### Pipeline 3 - train

In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import lib.database_module as dm
import lib.encoding_module as enc
import lib.wiki_module as wiki

### Mine: Select document vectors for all pages from database

In [3]:
page_vectors = dm.select_all_page_vectors()

Connected to server joshuacook.me.


In [4]:
page_ids = [tup[0] for tup in page_vectors]
page_vectors = [tup[1] for tup in page_vectors]

In [12]:
page_vectors_df = pd.DataFrame(page_vectors, index=page_ids)
page_vectors_df.head()

0         1         2         3         4         5    \
4341789   0.092992 -0.040985  0.006952  0.000243  0.001681  0.004999   
48201744  0.021301  0.027559 -0.003917 -0.010877  0.013055  0.000934   
2514975   0.124983  0.153025  0.026264 -0.257191 -0.210183  0.074847   
35135520  0.191357 -0.039003 -0.006018  0.006389 -0.001106 -0.004748   
27303975  0.034292  0.039499 -0.006095 -0.015156  0.026112 -0.039749   

               6         7         8         9      ...          490  \
4341789   0.003430  0.003523 -0.005129 -0.001911    ...    -0.021003   
48201744  0.000128  0.010317  0.001444 -0.009792    ...    -0.055040   
2514975   0.003862 -0.009666 -0.065705 -0.280972    ...    -0.022703   
35135520  0.015829  0.005612  0.011846  0.028248    ...     0.000578   
27303975 -0.081440 -0.036546  0.005076  0.018881    ...     0.030056   

               491       492       493       494       495       496  \
4341789  -0.039528  0.027716  0.021033 -0.034768 -0.022294  0.031537   
48201744  0.000110  0.057817  0.005156  0.058477  0.008807  0.043527   
2514975   0.015894 -0.002476  0.004992 -0.006653 -0.017394 -0.008525   
35135520 -0.004204  0.030916  0.010484 -0.012454  0.026050  0.038744   
27303975 -0.026965 -0.002426  0.042282  0.024082  0.004498 -0.009517   

               497       498       499  
4341789  -0.009710 -0.010331  0.025944  
48201744 -0.011693  0.064738  0.051296  
2514975   0.017188 -0.001596  0.000120  
35135520  0.015332 -0.008053  0.009506  
27303975 -0.005315 -0.003784 -0.010202  

[5 rows x 500 columns]

### Mine: Select category ids corresponding to pages from database

In [22]:
### GETTING THE Y
category_id_list = []

for page_id in page_vectors_df.index:
    category_info = dm.select_categories_for_page(page_id)
    category_id_list.append(category_info[0][1])

Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server joshuacook.me.
Connected to server 

In [29]:
len(category_id_list)

2270

In [30]:
X = page_vectors_df
y = category_id_list

In [32]:
print X.shape
print len(y)

(2270, 500)
2270


### Refine: Create a data dictionary with training and testing sets

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

data_dictionary = {
    'X' : X,
    'y' : y,
    'X_train' : X_train,
    'y_train' : y_train,
    'X_test' : X_test,
    'y_test' : y_test
}

In [35]:
data_dictionary

{'X':                0         1         2         3         4         5    \
 4341789   0.092992 -0.040985  0.006952  0.000243  0.001681  0.004999   
 48201744  0.021301  0.027559 -0.003917 -0.010877  0.013055  0.000934   
 2514975   0.124983  0.153025  0.026264 -0.257191 -0.210183  0.074847   
 35135520  0.191357 -0.039003 -0.006018  0.006389 -0.001106 -0.004748   
 27303975  0.034292  0.039499 -0.006095 -0.015156  0.026112 -0.039749   
 24626     0.250827 -0.105240  0.018797  0.017226  0.002876  0.007215   
 11313214  0.242895 -0.097608  0.006373  0.018134 -0.005170 -0.000981   
 34709570  0.023943  0.044653  0.015528 -0.024570  0.133896  0.149742   
 548937    0.235768 -0.090194  0.021657  0.020176 -0.009021  0.014186   
 23607991  0.249218 -0.146052  0.016309  0.034240 -0.005454  0.005678   
 974924    0.030096  0.040376  0.004332 -0.004615  0.025556 -0.024880   
 4472911   0.051218  0.048213 -0.024717  0.001998  0.007818 -0.012747   
 188496    0.050928  0.057854 -0.024186 -0.005

### Model: Fit, Score, and Tune a multi-label classification model

In [37]:
from sklearn.preprocessing import StandardScaler
scaled = StandardScaler()
scaled.fit_transform(data_dictionary['X_train'])

array([[ 1.28610864, -1.66381726,  0.10582142, ..., -0.74768823,
         1.50889971, -0.09604832],
       [-0.7518655 ,  0.1845184 ,  0.20843152, ..., -0.04666102,
        -0.09664746,  2.8446482 ],
       [ 0.69244071, -1.21869521,  0.08275493, ...,  0.45819665,
         0.19194252,  0.96672962],
       ..., 
       [-0.84918774, -0.21085435,  0.09265831, ..., -0.02955253,
        -1.50176391,  1.42269315],
       [-0.88622962,  0.07716258,  0.18339678, ...,  1.34135519,
         0.55237254, -0.11407188],
       [ 0.18194081,  1.01091549, -0.34031106, ...,  1.52136181,
         0.43621285,  0.56480066]])

In [38]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(data_dictionary['X_train'], data_dictionary['y_train'])
train_score = rf.score(data_dictionary['X_train'], data_dictionary['y_train'])
test_score = rf.score(data_dictionary['X_test'], data_dictionary['y_test'])
print "Train Score: ", train_score 
print "Test Score: ", test_score

Train Score:  0.997649823737
Test Score:  0.867957746479


### Present: Pickle tuned model for later use

In [39]:
from sklearn.externals import joblib

In [40]:
joblib.dump(rf, "the_pickled_model.pickle")

['the_pickled_model.pickle']